In [129]:
import matplotlib.pyplot as plt

import numpy as np #importa a biblioteca usada para trabalhar com vetores e matrizes
import pandas as pd #importa a biblioteca usada para trabalhar com dataframes
import util
import scipy
import scipy.optimize

#importa o arquivo e extrai as features
Xfeatures, Y = util.extract_features('datasets/everything.csv')
print(Xfeatures.shape)

(2772, 3323)


In [163]:
# parametros a serem utilizados neste exercicio
input_layer_size  = Xfeatures.shape[1]  # 20x20 dimensao das imagens de entrada
hidden_layer_size = 25   # 25 neuronios na camada oculta
num_labels = 2          # 10 rotulos, de 1 a 10  
                         #  (observe que a classe "0" recebe o rotulo 10)
    
print('\nCarregando parametros salvos da rede neural...\n')

# carregando os pesos da camada 1
Theta1 = np.random.rand(hidden_layer_size, input_layer_size+1)*np.sqrt(1/(input_layer_size+hidden_layer_size))

# carregando os pesos da camada 2
Theta2 = np.random.rand(num_labels, hidden_layer_size+1)*np.sqrt(1/(hidden_layer_size+num_labels))

# concatena os pesos em um único vetor
nn_params = np.concatenate([np.ravel(Theta1), np.ravel(Theta2)])

print('Pesos carregados com sucesso!')
print(Theta1.shape)
print(Theta2.shape)
print(nn_params.shape)


Carregando parametros salvos da rede neural...

Pesos carregados com sucesso!
(25, 3324)
(2, 26)
(83152,)


In [164]:
def sigmoid(z):
    """
    Calcula a função sigmoidal  
    """

    z = 1/(1+np.exp(-z))
    
    return z

In [165]:
def funcaoCusto(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y):
    '''
    Implementa a funcao de custo para a rede neural com duas camadas
    voltada para tarefa de classificacao
    
    Calcula o custo e gradiente da rede neural. 
    Os parametros da rede neural sao colocados no vetor nn_params
    e precisam ser transformados de volta nas matrizes de peso.
    
    input_layer_size - tamanho da camada de entrada
    hidden_layer_size - tamanho da camada oculta
    num_labels - numero de classes possiveis
    
    O vetor grad de retorno contem todas as derivadas parciais
    da rede neural.
    '''

    # Extrai os parametros de nn_params e alimenta as variaveis Theta1 e Theta2.
    Theta1 = np.reshape( nn_params[0:hidden_layer_size*(input_layer_size + 1)], (hidden_layer_size, input_layer_size+1) )
    Theta2 = np.reshape( nn_params[ hidden_layer_size*(input_layer_size + 1):], (num_labels, hidden_layer_size+1) )

    # Qtde de amostras
    m = X.shape[0]
         
    # A variavel a seguir precisa ser retornada corretamente
    J = 0;
    

    ########################## COMPLETE O CÓDIGO AQUI  ########################
    # Instrucoes: Voce deve completar o codigo a partir daqui 
    #               acompanhando os seguintes passos.
    #
    # (1): Lembre-se de transformar os rotulos Y em vetores com 10 posicoes,
    #         onde tera zero em todas posicoes exceto na posicao do rotulo
    #
    # (2): Execute a etapa de feedforward e coloque o custo na variavel J.
    #
    
    def l2a(i):
        a = np.zeros(num_labels)
        a[i] = 1
        return a

    Y = np.array([l2a(i) for i in y])
    
    a1 = np.insert(X.T, 0, 1,axis=0)
    z2 = np.matmul(Theta1, a1)
    a2 = np.insert(sigmoid(z2), 0, 1, axis=0)
    z3 = np.matmul(Theta2, a2)
    a3 = sigmoid(z3)

    J = 1/m * np.sum(np.sum(-Y * np.log(a3.T) - (1 - Y) * np.log(1 - a3.T)))
    
    
    
    
    
    
    
    
    
    
    
    

    ##########################################################################

    return J


print('\nFuncao de custo sem regularizacao ...\n')

J = funcaoCusto(nn_params, input_layer_size, hidden_layer_size, num_labels, Xfeatures, Y)

print('Custo com os parametros (gerados aleatoriamente segundo inicialização de Xavier): %1.6f ' %J)


Funcao de custo sem regularizacao ...

Custo com os parametros (gerados aleatoriamente segundo inicialização de Xavier): 1.827966 


In [166]:
def funcaoCusto_reg(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, vLambda):
    '''
    Implementa a funcao de custo para a rede neural com duas camadas
    voltada para tarefa de classificacao
    
    Calcula o custo e gradiente da rede neural. 
    Os parametros da rede neural sao colocados no vetor nn_params
    e precisam ser transformados de volta nas matrizes de peso.
    
    input_layer_size - tamanho da camada de entrada
    hidden_layer_size - tamanho da camada oculta
    num_labels - numero de classes possiveis
    lambda - parametro de regularizacao
    
    O vetor grad de retorno contem todas as derivadas parciais
    da rede neural.
    '''

    # Extrai os parametros de nn_params e alimenta as variaveis Theta1 e Theta2.
    Theta1 = np.reshape( nn_params[0:hidden_layer_size*(input_layer_size + 1)], (hidden_layer_size, input_layer_size+1) )
    Theta2 = np.reshape( nn_params[ hidden_layer_size*(input_layer_size + 1):], (num_labels, hidden_layer_size+1) )

    # Qtde de amostras
    m = X.shape[0]
         
    # A variavel a seguir precisa ser retornada corretamente
    J = 0;
    

    ########################## COMPLETE O CÓDIGO AQUI  ########################
    # Instrucoes: Voce deve completar o codigo a partir daqui 
    #               acompanhando os seguintes passos.
    #
    # (1): Lembre-se de transformar os rotulos Y em vetores com 10 posicoes,
    #         onde tera zero em todas posicoes exceto na posicao do rotulo
    #
    # (2): Execute a etapa de feedforward e coloque o custo na variavel J.
    #
    #
    # (3): Implemente a regularização na função de custo.
    #

    reg = vLambda / (2*m) * (np.sum(Theta1[:,1:] ** 2) + np.sum(Theta2[:,1:] ** 2))
    J = funcaoCusto(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y) + reg
    
    
    
    
    
    
    
    
    
    
    
    
    

    ##########################################################################

    return J

In [167]:
print('\nChecando a funcao de custo (c/ regularizacao) ... \n')

# Parametro de regularizacao dos pesos (aqui sera igual a 1).
vLambda = 1;

J = funcaoCusto_reg(nn_params, input_layer_size, hidden_layer_size, num_labels, Xfeatures, Y, vLambda)

print('Custo com os parametros (inicializados aleatoriamente): %1.6f ' %J)


Checando a funcao de custo (c/ regularizacao) ... 

Custo com os parametros (inicializados aleatoriamente): 1.829567 


In [168]:
def inicializaPesosAleatorios(L_in, L_out, randomSeed = None):
    '''
    Inicializa aleatoriamente os pesos de uma camada usando 
    L_in (conexoes de entrada) e L_out (conexoes de saida).

    W sera definido como uma matriz de dimensoes [L_out, 1 + L_in]
    visto que devera armazenar os termos para "bias".
    
    randomSeed: indica a semente para o gerador aleatorio
    '''

    epsilon_init = 0.12
    
    # se for fornecida uma semente para o gerador aleatorio
    if randomSeed is not None:
        W = np.random.RandomState(randomSeed).rand(L_out, 1 + L_in) * 2 * epsilon_init - epsilon_init
        
    # se nao for fornecida uma semente para o gerador aleatorio
    else:
        W = np.random.rand(L_out, 1 + L_in) * 2 * epsilon_init - epsilon_init
        
    return W


print('\nInicializando parametros da rede neural...\n')
    
#initial_Theta1 = inicializaPesosAleatorios(input_layer_size, hidden_layer_size, randomSeed = 10)
#initial_Theta2 = inicializaPesosAleatorios(hidden_layer_size, num_labels, randomSeed = 20)

# junta os pesos iniciais em um unico vetor
#initial_rna_params = np.concatenate([np.ravel(initial_Theta1), np.ravel(initial_Theta2)])
initial_rna_params = nn_params


Inicializando parametros da rede neural...



In [169]:
def sigmoidGradient(z):
    '''
    Retorna o gradiente da funcao sigmoidal para z 
    
    Calcula o gradiente da funcao sigmoidal
    para z. A funcao deve funcionar independente se z for matriz ou vetor.
    Nestes casos,  o gradiente deve ser calculado para cada elemento.
    '''
    
    g = np.zeros(z.shape)

    ########################## COMPLETE O CÓDIGO AQUI  ########################
    # Instrucoes: Calcula o gradiente da funcao sigmoidal para 
    #           cada valor de z (seja z matriz, escalar ou vetor).
    #

    g = sigmoid(z) * (1 - sigmoid(z))
    
    
    
    
    
    
    
    
    
    
    
    
        
    ##########################################################################

    return g

print('\nAvaliando o gradiente da sigmoide...\n')

g = sigmoidGradient(np.array([1,-0.5, 0, 0.5, 1]))
print('Gradiente da sigmoide avaliado em [1 -0.5 0 0.5 1]:\n')
print(g)


Avaliando o gradiente da sigmoide...

Gradiente da sigmoide avaliado em [1 -0.5 0 0.5 1]:

[0.19661193 0.23500371 0.25       0.23500371 0.19661193]


In [170]:
def funcaoCusto_backp(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y):
    '''
    Implementa a funcao de custo para a rede neural com tres camadas
    voltada para a tarefa de classificacao
    
    Calcula o custo e gradiente da rede neural. 
    Os parametros da rede sao colocados no vetor nn_params
    e precisam ser transformados de volta nas matrizes de peso.
    
    input_layer_size - tamanho da camada de entrada
    hidden_layer_size - tamanho da camada oculta
    num_labels - numero de classes possiveis
    lambda - parametro de regularizacao
    
    O vetor grad de retorno contem todas as derivadas parciais
    da rede neural.
    '''

    # Extrai os parametros de nn_params e alimenta as variaveis Theta1 e Theta2.
    Theta1 = np.reshape( nn_params[0:hidden_layer_size*(input_layer_size + 1)], (hidden_layer_size, input_layer_size+1) )
    Theta2 = np.reshape( nn_params[ hidden_layer_size*(input_layer_size + 1):], (num_labels, hidden_layer_size+1) )

    # Qtde de amostras
    m = X.shape[0]
         
    # As variaveis a seguir precisam ser retornadas corretamente
    J = 0;
    Theta1_grad = np.zeros(Theta1.shape)
    Theta2_grad = np.zeros(Theta2.shape)
    

    ########################## COMPLETE O CÓDIGO AQUI  ########################
    # Instrucoes: Voce deve completar o codigo a partir daqui 
    #               acompanhando os seguintes passos.
    #
    # (1): Lembre-se de transformar os rotulos Y em vetores com 10 posicoes,
    #         onde tera zero em todas posicoes exceto na posicao do rotulo
    #
    # (2): Execute a etapa de feedforward e coloque o custo na variavel J.
    #
    # (3): Implemente o algoritmo de backpropagation para calcular 
    #      os gradientes e alimentar as variaveis Theta1_grad e Theta2_grad.
    #
    #

    def l2a(i):
        a = np.zeros(num_labels)
        a[i] = 1
        return a

    Y = np.array([l2a(i) for i in y])
    
    a1 = np.insert(X.T, 0, 1,axis=0)
    z2 = np.matmul(Theta1, a1)
    a2 = np.insert(sigmoid(z2), 0, 1, axis=0)
    z3 = np.matmul(Theta2, a2)
    a3 = sigmoid(z3)

    J = 1/m * np.sum(np.sum(-Y * np.log(a3.T) - (1 - Y) * np.log(1 - a3.T)))

    d3 = a3 - Y.T
    d2 = np.multiply(np.matmul(Theta2[:,1:].T, d3), sigmoidGradient(z2))

    Theta1_grad = 1 / m * np.matmul(d2, a1.T)
    Theta2_grad = 1 / m * np.matmul(d3, a2.T)
    
    
    
    
    
    
    
    
    
    
    
    
    

    ##########################################################################

    # Junta os gradientes
    grad = np.concatenate([np.ravel(Theta1_grad), np.ravel(Theta2_grad)])

    return J, grad

In [171]:
def funcaoCusto_backp_reg(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, vLambda):
    '''
    Implementa a funcao de custo para a rede neural com tres camadas
    voltada para tarefa de classificacao
    
    Calcula o custo e gradiente da rede neural. 
    Os parametros da rede neural sao colocados no vetor nn_params
    e precisam ser transformados de volta nas matrizes de peso.
    
    input_layer_size - tamanho da camada de entrada
    hidden_layer_size - tamanho da camada oculta
    num_labels - numero de classes possiveis
    lambda - parametro de regularizacao
    
    O vetor grad de retorno contem todas as derivadas parciais
    da rede neural.
    '''

    # Extrai os parametros de nn_params e alimenta as variaveis Theta1 e Theta2.
    Theta1 = np.reshape( nn_params[0:hidden_layer_size*(input_layer_size + 1)], (hidden_layer_size, input_layer_size+1) )
    Theta2 = np.reshape( nn_params[ hidden_layer_size*(input_layer_size + 1):], (num_labels, hidden_layer_size+1) )

    # Qtde de amostras
    m = X.shape[0]
         
    # As variaveis a seguir precisam ser retornadas corretamente
    J = 0;
    Theta1_grad = np.zeros(Theta1.shape)
    Theta2_grad = np.zeros(Theta2.shape)
    

    ########################## COMPLETE O CÓDIGO AQUI  ########################
    # Instrucoes: Voce deve completar o codigo a partir daqui 
    #               acompanhando os seguintes passos.
    #
    # (1): Lembre-se de transformar os rotulos Y em vetores com 10 posicoes,
    #         onde tera zero em todas posicoes exceto na posicao do rotulo
    #
    # (2): Execute a etapa de feedforward e coloque o custo na variavel J.
    #
    # (3): Implemente o algoritmo de backpropagation para calcular 
    #      os gradientes e alimentar as variaveis Theta1_grad e Theta2_grad.
    #
    # (4): Implemente a regularização na função de custo e gradiente.
    #

    def l2a(i):
        a = np.zeros(num_labels)
        a[i] = 1
        return a

    Y = np.array([l2a(i) for i in y])
    
    a1 = np.insert(X.T, 0, 1,axis=0)
    z2 = np.matmul(Theta1, a1)
    a2 = np.insert(sigmoid(z2), 0, 1, axis=0)
    z3 = np.matmul(Theta2, a2)
    a3 = sigmoid(z3)
    
    reg = vLambda / (2*m) * (np.sum(Theta1[:,1:] ** 2) + np.sum(Theta2[:,1:] ** 2))
    J = 1/m * np.sum(np.sum(-Y * np.log(a3.T) - (1 - Y) * np.log(1 - a3.T))) + reg

    d3 = a3 - Y.T
    d2 = np.multiply(np.matmul(Theta2[:,1:].T, d3), sigmoidGradient(z2))

    Theta1_grad = 1 / m * np.matmul(d2, a1.T)
    Theta2_grad = 1 / m * np.matmul(d3, a2.T)

    reg1 = (vLambda / m) * Theta1
    reg1[:,0] = 0
    reg2 = (vLambda / m) * Theta2
    reg2[:,0] = 0
    
    Theta1_grad = Theta1_grad + reg1
    Theta2_grad = Theta2_grad + reg2
    
    
    
    
    
    
    
    
    
    
    
    
    

    ##########################################################################

    # Junta os gradientes
    grad = np.concatenate([np.ravel(Theta1_grad), np.ravel(Theta2_grad)])

    return J, grad




# Parametro de regularizacao dos pesos.
vLambda = 3;


print('\n\nChecando a funcao de custo (c/ regularizacao) ... \n')

J, grad = funcaoCusto_backp_reg(nn_params, input_layer_size, hidden_layer_size, num_labels, Xfeatures, Y, vLambda)

print('Custo com os parametros (carregados do arquivo): %1.6f' %J)



Checando a funcao de custo (c/ regularizacao) ... 

Custo com os parametros (carregados do arquivo): 1.832769


In [172]:
print('\nTreinando a rede neural.......')
print('.......(Aguarde, pois esse processo por ser um pouco demorado.)\n')

# Apos ter completado toda a tarefa, mude o parametro MaxIter para
# um valor maior e verifique como isso afeta o treinamento.
MaxIter = 500

# Voce tambem pode testar valores diferentes para lambda.
vLambda = 1

# Minimiza a funcao de custo
result = scipy.optimize.minimize(fun=funcaoCusto_backp_reg, x0=initial_rna_params, args=(input_layer_size, hidden_layer_size, num_labels, Xfeatures, Y, vLambda),  
                method='TNC', jac=True, options={'maxiter': MaxIter})

# Coleta os pesos retornados pela função de minimização
nn_params = result.x

# Obtem Theta1 e Theta2 back a partir de rna_params
Theta1 = np.reshape( nn_params[0:hidden_layer_size*(input_layer_size + 1)], (hidden_layer_size, input_layer_size+1) )
Theta2 = np.reshape( nn_params[ hidden_layer_size*(input_layer_size + 1):], (num_labels, hidden_layer_size+1) )

print(result)


Treinando a rede neural.......
.......(Aguarde, pois esse processo por ser um pouco demorado.)



D:\Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:61: RuntimeWarning: divide by zero encountered in log
D:\Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:61: RuntimeWarning: invalid value encountered in multiply


     fun: 0.3738408220742199
     jac: array([ 2.30038215e-03, -3.59494724e-06, -4.18419351e-06, ...,
        3.06689998e-03,  2.98617621e-03,  2.91761644e-03])
 message: 'Linear search failed'
    nfev: 141
     nit: 10
  status: 4
 success: False
       x: array([ 0.54455321, -0.08321874,  0.02049245, ...,  0.70778503,
        1.48267727, -3.43707206])


In [174]:
def predicao(Theta1, Theta2, X):
    '''
    Prediz o rotulo de uma amostra apresentada a rede neural
    
    Prediz o rotulo de X ao utilizar
    os pesos treinados na rede neural (Theta1, Theta2)
    '''
    
    m = X.shape[0] # número de amostras
    num_labels = Theta2.shape[0]
    
    p = np.zeros(m)

    a1 = np.hstack( [np.ones([m,1]),X] )
    h1 = sigmoid( np.dot(a1,Theta1.T) )

    a2 = np.hstack( [np.ones([m,1]),h1] ) 
    h2 = sigmoid( np.dot(a2,Theta2.T) )
    
    p = np.argmax(h2,axis=1)
    #p = p+1
    
    return p
    

pred = predicao(Theta1, Theta2, Xfeatures)
print(pred[:5])
print(Y[:5])

print('\nAcuracia no conjunto de treinamento: %f\n'%( np.mean( pred == Y ) * 100) )

[1 0 0 1 0]
[1 0 0 1 0]

Acuracia no conjunto de treinamento: 95.093795



In [176]:
pd.DataFrame(Theta1).to_csv('pesos_Theta1.csv', index=False)
pd.DataFrame(Theta2).to_csv('pesos_Theta2.csv', index=False)